# Calcul de la consommaton globale

On commence par charger les données, on les affiche, on les trie et on fait une première synthèse des consommation par moteur et par vol.


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import tabata as tbt
import matplotlib.pyplot as plt
%matplotlib inline